<a href="https://colab.research.google.com/github/mr-44-bs/createLyrics/blob/main/M_fineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers==4.20.1
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.7 MB/s eta 0:00:00


In [ ]:
from transformers import T5Tokenizer,AutoModelForCausalLM
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-small")

Downloading:   0%|          | 0.00/787k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/846 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [ ]:
import pandas as pd
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lyrics/MrChildren_lyrics.csv",index_col=0).reset_index()

In [ ]:
import re
texts = []

for i in range(len(df)):

    text = df.iloc[i,1]
    text = re.sub("【[^】]+】","",text)
    text = re.sub("[＜＞]","",text)
    text = re.sub("[■]","",text)
    text = text.split("。")
    texts.append("。\n".join(text))

train_data_path = "train.text"
with open(train_data_path,"w") as f:
    f.writelines(texts)

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

#データセットの設定
train_dataset = TextDataset(
    tokenizer = tokenizer,
    file_path = train_data_path,
    block_size = 300 #文章の長さを揃える必要がある
)

#データの入力に関する設定
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm= False
)

# 訓練に関する設定
training_args = TrainingArguments(
    output_dir="./gpt2-ft",  # 関連ファイルを保存するパス
    overwrite_output_dir=True,  # ファイルを上書きするかどうか
    num_train_epochs=3,  # エポック数
    per_device_train_batch_size=8,  # バッチサイズ
    logging_steps=100,  # 途中経過を表示する間隔
    save_steps=800  # モデルを保存する間隔
)

#トレーナーの設定
trainer = Trainer(
    model =model,
    args=training_args,
    data_collator = data_collator,
    train_dataset = train_dataset
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
%%time
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 218
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 84


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 44.2 s, sys: 401 ms, total: 44.6 s
Wall time: 50.1 s


TrainOutput(global_step=84, training_loss=4.049105689639137, metrics={'train_runtime': 50.0431, 'train_samples_per_second': 13.069, 'train_steps_per_second': 1.679, 'total_flos': 100127923200000.0, 'train_loss': 4.049105689639137, 'epoch': 3.0})

In [ ]:
import pickle

In [ ]:
with open("/content/gdrive/My Drive/Colab Notebooks/lyrics/model0512_2.pkl","wb") as f:
    pickle.dump(model,f)

In [ ]:
with open("/content/gdrive/My Drive/Colab Notebooks/lyrics/model0512_2.pkl", "rb") as f:
    saved_model = pickle.load(f)

In [ ]:
def getarate_sentences(seed_sentence,saved_model):
    x = tokenizer.encode(seed_sentence, return_tensors="pt",
    add_special_tokens=False)  # 入力
    x = x.cuda()  # GPU対応
    y = model.generate(x, #入力
                       min_length=50,  # 文章の最小長
                       max_length=100,  # 文章の最大長
                       do_sample=True,   # 次の単語を確率で選ぶ
                       top_k=50, # Top-Kサンプリング
                       top_p=0.95,  # Top-pサンプリング
                       temperature=1.2,  # 確率分布の調整
                       num_return_sequences=3,  # 生成する文章の数
                       pad_token_id=tokenizer.pad_token_id,  # パディングのトークンID
                       bos_token_id=tokenizer.bos_token_id,  # テキスト先頭のトークンID
                       eos_token_id=tokenizer.eos_token_id,  # テキスト終端のトークンID
                       bad_word_ids=[[tokenizer.unk_token_id]]  # 生成が許可されないトークンID
                       )
    generated_sentences = tokenizer.batch_decode(y, skip_special_tokens=True)  # 特殊トークンをスキップして文章に変換
    return generated_sentences

In [ ]:
seed_sentence = "待ちわびてた約束の週末"
generated_sentences = getarate_sentences(seed_sentence,saved_model)  # 生成された文章
for sentence in generated_sentences:
    print(sentence)

待ちわびてた約束の週末が来てくれました - 僕が 夢を叶えるために 求めていたものとは... おひさしぶりです!いつも見てますよ!待ってますよね笑ってる姿が大好きです!!もう少し早く撮れっっ放しでよよ 夢中なのに足がふらふらふら降ろすもやし座の星空見てたら 君も「僕も」
待ちわびてた約束の週末なんて、今更だけど心待ちにしてた大切な想いを忘れかけないでもうひとつの大切な想いを想うだけ愛しい人想そんな想いは今も胸を突き刺せるけれど だからこそあの日の夜まで待ってたって忘れても夢でもそんなささいな話だったらどうかな きっといつか君の声の聞こえたような気がする ずっとそうずっとhobby oh 僕を忘れないでもうひとつの大切な想い
待ちわびてた約束の週末 1人で 寂しさを包んだ笑いじゃくる手が届かない夢の向こうへ 何度となく夢が膨らみ愛し合う2人が 明日 あの日まで想い 感じていられるでしょうか いつも優しく包みながらいつまでも いつか 恋しくなる夜が くるどんなにつらくても愛 生きていきなさい いつも愛しあわせの夢は叶うから君に いつか
